In [1]:
# from collections import defaultdict
from pyomo.util.infeasible import log_infeasible_constraints
from pyomo.environ import *
from pyomo.opt import SolverFactory

import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

import plotly.express as px
import plotly.graph_objects as go

from collections import OrderedDict
from collections import OrderedDict

import csv
import math

In [2]:
nonFixedTasks = pd.read_csv('Data/Day1/NonFixedTasks.csv', index_col=False)
nonFixedTasks['Start']=(pd.to_datetime(nonFixedTasks['Start'].str.strip(), format='%I:%M:%S %p')).dt.time
nonFixedTasks['End']=(pd.to_datetime(nonFixedTasks['End'].str.strip(), format='%I:%M:%S %p')).dt.time
nonFixedTasks

,Task,Start,End,Duration
0,Admin Ordering,09:00:00,16:00:00,1
1,Admin StayAtHome,09:00:00,16:00:00,1


In [3]:
columns = ['Task', 'Start', 'End', 'Slots']
nonFixedTasksWithSlots = nonFixedTasks


for index, row in nonFixedTasksWithSlots.iterrows():
    nonFixedTasksWithSlots.at[index,'Duration'] = nonFixedTasksWithSlots.at[index,'Duration']*4

nonFixedTasksWithSlots.columns = columns
nonFixedTasksWithSlots = [tuple(x) for x in nonFixedTasksWithSlots.to_numpy()]
nonFixedTasksWithSlots

[('Admin Ordering', datetime.time(9, 0), datetime.time(16, 0), 4),
 ('Admin StayAtHome', datetime.time(9, 0), datetime.time(16, 0), 4)]

In [4]:
EmpTasks1 = pd.read_csv('Data/Day1/EmpTasks.csv', index_col=False)
EmpTasks1 = [tuple(x) for x in EmpTasks1.to_numpy()]
EmpTasks1

[('15/11/2018', 'Employee501603', 'Downtown Platform'),
 ('15/11/2018', 'Employee501603', 'MSR Opener Platform'),
 ('15/11/2018', 'Employee501603', 'MSR Closer Platform'),
 ('15/11/2018', 'Employee501715', 'Downtown Lobby'),
 ('15/11/2018', 'Employee501715', 'MSR Opener Lobby'),
 ('15/11/2018', 'Employee501715', 'MSR Closer Lobby'),
 ('15/11/2018', 'Employee501716', 'Downtown Lobby'),
 ('15/11/2018', 'Employee501716', 'Teller Closer Lobby'),
 ('15/11/2018', 'Employee501721', 'Downtown Platform'),
 ('15/11/2018', 'Employee501721', 'MSR Opener Platform'),
 ('15/11/2018', 'Employee501721', 'MSR Closer Platform'),
 ('15/11/2018', 'Employee501721', 'BM Admin Time'),
 ('15/11/2018', 'Employee501768', 'Downtown Lobby'),
 ('15/11/2018', 'Employee501768', 'Teller Closer Lobby'),
 ('15/11/2018', 'Employee501792', 'Downtown Platform'),
 ('15/11/2018', 'Employee501792', 'MSR Opener Platform'),
 ('15/11/2018', 'Employee501792', 'MSR Closer Platform'),
 ('15/11/2018', 'Employee501587', 'Downtown Pla

In [5]:
assignment_list = pd.read_csv('Data/Results/assignment_list.csv', index_col=False)
assignment_list = assignment_list[['Employee', 'Day', 'TSlot', 'Assignment']]
assignment_list = assignment_list.loc[assignment_list['Day']=='15/11/2018']
assignment_list = [tuple(x) for x in assignment_list.to_numpy()]
assignment_list

[('Employee501603', '15/11/2018', '09:00:00', 'Downtown Platform'),
 ('Employee501603', '15/11/2018', '09:15:00', 'Downtown Platform'),
 ('Employee501603', '15/11/2018', '09:30:00', 'Downtown Platform'),
 ('Employee501603', '15/11/2018', '09:45:00', 'Downtown Platform'),
 ('Employee501603', '15/11/2018', '10:00:00', 'Downtown Platform'),
 ('Employee501603', '15/11/2018', '10:15:00', 'Downtown Platform'),
 ('Employee501603', '15/11/2018', '10:30:00', 'Downtown Platform'),
 ('Employee501603', '15/11/2018', '10:45:00', 'Downtown Platform'),
 ('Employee501603', '15/11/2018', '11:00:00', 'Downtown Platform'),
 ('Employee501603', '15/11/2018', '11:15:00', 'Downtown Platform'),
 ('Employee501603', '15/11/2018', '11:30:00', 'Downtown Platform'),
 ('Employee501603', '15/11/2018', '11:45:00', 'Downtown Platform'),
 ('Employee501603', '15/11/2018', '12:00:00', 'Downtown Platform'),
 ('Employee501603', '15/11/2018', '12:15:00', 'Downtown Platform'),
 ('Employee501603', '15/11/2018', '12:30:00', 'D

In [6]:
schedule = pd.read_csv('Data/Results/schedule.csv')
schedule = schedule.iloc[:,[0,1]]
schedule.columns = ['Time', '15/11/2018']
schedule = [tuple(x) for x in schedule.to_numpy()]

schedule

[('09:00:00',
  "{'BM Admin Time': ['Employee501721'], 'ABM Admin Time': [], 'Downtown Lobby': ['Employee501715', 'Employee501716', 'Employee501768', 'Employee501587'], 'Downtown Platform': ['Employee501603', 'Employee501792', 'Employee501595']}"),
 ('09:15:00',
  "{'BM Admin Time': ['Employee501721'], 'ABM Admin Time': [], 'Downtown Lobby': ['Employee501715', 'Employee501716', 'Employee501768', 'Employee501587'], 'Downtown Platform': ['Employee501603', 'Employee501792', 'Employee501595']}"),
 ('09:30:00',
  "{'BM Admin Time': ['Employee501721'], 'ABM Admin Time': [], 'Downtown Lobby': ['Employee501715', 'Employee501716', 'Employee501768', 'Employee501587'], 'Downtown Platform': ['Employee501603', 'Employee501792', 'Employee501595']}"),
 ('09:45:00',
  "{'BM Admin Time': ['Employee501721'], 'ABM Admin Time': [], 'Downtown Lobby': ['Employee501715', 'Employee501716', 'Employee501768', 'Employee501587'], 'Downtown Platform': ['Employee501603', 'Employee501792', 'Employee501595']}"),
 ('1

In [7]:
import ast

columns = ['Time', 'Task', 'Scheduled']
allocations = pd.DataFrame()

for index, value in schedule:
    dic = ast.literal_eval(value)
    for Task, Employees in dic.items():
        # print(index, Task, len(Employees))
        allocations = allocations.append(pd.Series([index, Task, len(Employees)]), ignore_index=True)

allocations.columns = columns
allocations = [tuple(x) for x in allocations.to_numpy()]
allocations

[('09:00:00', 'BM Admin Time', 1.0),
 ('09:00:00', 'ABM Admin Time', 0.0),
 ('09:00:00', 'Downtown Lobby', 4.0),
 ('09:00:00', 'Downtown Platform', 3.0),
 ('09:15:00', 'BM Admin Time', 1.0),
 ('09:15:00', 'ABM Admin Time', 0.0),
 ('09:15:00', 'Downtown Lobby', 4.0),
 ('09:15:00', 'Downtown Platform', 3.0),
 ('09:30:00', 'BM Admin Time', 1.0),
 ('09:30:00', 'ABM Admin Time', 0.0),
 ('09:30:00', 'Downtown Lobby', 4.0),
 ('09:30:00', 'Downtown Platform', 3.0),
 ('09:45:00', 'BM Admin Time', 1.0),
 ('09:45:00', 'ABM Admin Time', 0.0),
 ('09:45:00', 'Downtown Lobby', 4.0),
 ('09:45:00', 'Downtown Platform', 3.0),
 ('10:00:00', 'BM Admin Time', 1.0),
 ('10:00:00', 'ABM Admin Time', 0.0),
 ('10:00:00', 'BMT Meeting', 0.0),
 ('10:00:00', 'Downtown Lobby', 4.0),
 ('10:00:00', 'Downtown Platform', 3.0),
 ('10:15:00', 'BM Admin Time', 1.0),
 ('10:15:00', 'ABM Admin Time', 0.0),
 ('10:15:00', 'BMT Meeting', 0.0),
 ('10:15:00', 'Downtown Lobby', 3.0),
 ('10:15:00', 'Downtown Platform', 4.0),
 ('10:

In [8]:
TimeAssEmpReq1 = pd.read_csv('Data/Day1/Time_Ass_Req_Schedule.csv', index_col=False)
TimeAssEmpReq1['Time']=(pd.to_datetime(TimeAssEmpReq1['Time'].str.strip(), format='%I:%M:%S %p')).dt.time
TimeAssEmpReq1 = [tuple(x) for x in TimeAssEmpReq1.to_numpy()]
TimeAssEmpReq1

[('15/11/2018', datetime.time(9, 0), 'Downtown Lobby', 3.8, 2),
 ('15/11/2018', datetime.time(9, 15), 'Downtown Lobby', 3.2, 2),
 ('15/11/2018', datetime.time(9, 30), 'Downtown Lobby', 2.6, 2),
 ('15/11/2018', datetime.time(9, 45), 'Downtown Lobby', 2.8, 2),
 ('15/11/2018', datetime.time(10, 0), 'Downtown Lobby', 4.1, 2),
 ('15/11/2018', datetime.time(10, 15), 'Downtown Lobby', 2.1, 2),
 ('15/11/2018', datetime.time(10, 30), 'Downtown Lobby', 3.7, 2),
 ('15/11/2018', datetime.time(10, 45), 'Downtown Lobby', 3.3, 2),
 ('15/11/2018', datetime.time(11, 0), 'Downtown Lobby', 4.3, 2),
 ('15/11/2018', datetime.time(11, 15), 'Downtown Lobby', 4.4, 2),
 ('15/11/2018', datetime.time(11, 30), 'Downtown Lobby', 4.9, 2),
 ('15/11/2018', datetime.time(11, 45), 'Downtown Lobby', 3.3, 2),
 ('15/11/2018', datetime.time(12, 0), 'Downtown Lobby', 5.5, 2),
 ('15/11/2018', datetime.time(12, 15), 'Downtown Lobby', 4.5, 2),
 ('15/11/2018', datetime.time(12, 30), 'Downtown Lobby', 2.8, 2),
 ('15/11/2018', da

In [9]:
allocationsWithRequirement = pd.DataFrame()
columns = ['Day', 'TSlot', 'Task', 'Requirement', 'Scheduled', 'Gap']

for Day, Time, Task, Requirement, Scheduled in TimeAssEmpReq1:
    #print(Day, Time, Task, Requirement, Scheduled)
    Time = Time.strftime("%H:%M:%S")
    #print(Time, type(Time))
    for all_time, all_task, all_scheduled in allocations:
        #print(all_time, all_task, all_scheduled)
        if Time == all_time and Task == all_task:
            #print(type(Requirement), type(all_scheduled))
            allocationsWithRequirement = allocationsWithRequirement.append(pd.Series([Day, all_time, all_task, Requirement, all_scheduled, (Requirement-all_scheduled)]), ignore_index=True)
    
allocationsWithRequirement.columns = columns
allocationsWithRequirement = [tuple(x) for x in allocationsWithRequirement.to_numpy()]
allocationsWithRequirement

[('15/11/2018', '09:00:00', 'Downtown Lobby', 3.8, 4.0, -0.20000000000000018),
 ('15/11/2018', '09:15:00', 'Downtown Lobby', 3.2, 4.0, -0.7999999999999998),
 ('15/11/2018', '09:30:00', 'Downtown Lobby', 2.6, 4.0, -1.4),
 ('15/11/2018', '09:45:00', 'Downtown Lobby', 2.8, 4.0, -1.2000000000000002),
 ('15/11/2018', '10:00:00', 'Downtown Lobby', 4.1, 4.0, 0.09999999999999964),
 ('15/11/2018', '10:15:00', 'Downtown Lobby', 2.1, 3.0, -0.8999999999999999),
 ('15/11/2018', '10:30:00', 'Downtown Lobby', 3.7, 4.0, -0.2999999999999998),
 ('15/11/2018', '10:45:00', 'Downtown Lobby', 3.3, 4.0, -0.7000000000000002),
 ('15/11/2018', '11:00:00', 'Downtown Lobby', 4.3, 4.0, 0.2999999999999998),
 ('15/11/2018', '11:15:00', 'Downtown Lobby', 4.4, 4.0, 0.40000000000000036),
 ('15/11/2018', '11:30:00', 'Downtown Lobby', 4.9, 4.0, 0.9000000000000004),
 ('15/11/2018', '11:45:00', 'Downtown Lobby', 3.3, 4.0, -0.7000000000000002),
 ('15/11/2018', '12:00:00', 'Downtown Lobby', 5.5, 4.0, 1.5),
 ('15/11/2018', '1

In [10]:
from datetime import datetime
columns = ['Employee', 'TSlot', 'Task', 'RequiredTask']

timeSlotStatus = pd.DataFrame()
for Day, Employee, EligibleTask in EmpTasks1:
    for RequiredTask, Start, End, Slots in nonFixedTasksWithSlots:
        if EligibleTask == RequiredTask:
            print(EligibleTask)
            count = 0
            for ass_emp, ass_day, ass_tslot, ass_task in assignment_list:
                if ass_emp == Employee:
                    repeat = True
                    while repeat==True:
                        if count == 0:
                            day = datetime.strptime(ass_day, '%d/%m/%Y').date()
                            time = datetime.strptime(ass_tslot, '%H:%M:%S')
                            dayWithTime = datetime.combine(day, time.time())
                            # print(dayWithTime)
                            timeSlotStatus = timeSlotStatus.append(pd.Series([ass_emp, ass_tslot, ass_task, RequiredTask]), ignore_index=True)
                            # print(ass_tslot, ass_task)
                            prevTime = dayWithTime
                            count +=1
                            repeat=False
                        else:
                            next_slot = (prevTime + timedelta(minutes=15)).time().strftime("%H:%M:%S")
                            # print(type(ass_tslot), type(next_slot))
                            # print(ass_tslot, next_slot)
                            if ass_tslot==next_slot:
                                # print(ass_tslot+" slot not free")
                                day = datetime.strptime(ass_day, '%d/%m/%Y').date()
                                time = datetime.strptime(ass_tslot, '%H:%M:%S')
                                dayWithTime = datetime.combine(day, time.time())
                                timeSlotStatus = timeSlotStatus.append(pd.Series([ass_emp, ass_tslot, ass_task, RequiredTask]), ignore_index=True)
                                # print(ass_tslot, ass_task)
                                prevTime = dayWithTime
                                repeat=False
                            else:
                                dayWithTime = datetime.combine(day, (prevTime + timedelta(minutes=15)).time())
                                # print(dayWithTime)
                                timeSlotStatus = timeSlotStatus.append(pd.Series([ass_emp, dayWithTime.time().strftime("%H:%M:%S"), 'free', RequiredTask]), ignore_index=True)
                                # print(dayWithTime.time(), 'free')
                                prevTime = dayWithTime
                                repeat=True
                            
timeSlotStatus.columns = columns
timeSlotStatus = [tuple(x) for x in timeSlotStatus.to_numpy()]
timeSlotStatus

Admin Ordering
Admin StayAtHome


[('Employee501595', '09:00:00', 'Downtown Platform', 'Admin Ordering'),
 ('Employee501595', '09:15:00', 'Downtown Platform', 'Admin Ordering'),
 ('Employee501595', '09:30:00', 'Downtown Platform', 'Admin Ordering'),
 ('Employee501595', '09:45:00', 'Downtown Platform', 'Admin Ordering'),
 ('Employee501595', '10:00:00', 'Downtown Platform', 'Admin Ordering'),
 ('Employee501595', '10:15:00', 'Downtown Platform', 'Admin Ordering'),
 ('Employee501595', '10:30:00', 'Downtown Platform', 'Admin Ordering'),
 ('Employee501595', '10:45:00', 'Downtown Platform', 'Admin Ordering'),
 ('Employee501595', '11:00:00', 'Downtown Platform', 'Admin Ordering'),
 ('Employee501595', '11:15:00', 'Downtown Platform', 'Admin Ordering'),
 ('Employee501595', '11:30:00', 'Downtown Platform', 'Admin Ordering'),
 ('Employee501595', '11:45:00', 'Downtown Platform', 'Admin Ordering'),
 ('Employee501595', '12:00:00', 'Downtown Platform', 'Admin Ordering'),
 ('Employee501595', '12:15:00', 'Downtown Platform', 'Admin Orde

In [11]:
timeSlotStatusWithGap = pd.DataFrame()
for Employee, TSlot, Task, RequiredTask in timeSlotStatus:
    prevTSlot=''
    for Day, ass_tslot, ass_task, requirement, scheduled, gap in allocationsWithRequirement:
        if TSlot==ass_tslot and Task==ass_task:
        # print(Day, ass_tslot, Task, requirement, scheduled, gap)
            timeSlotStatusWithGap = timeSlotStatusWithGap.append(pd.Series([Employee, ass_tslot, Task, gap, RequiredTask]), ignore_index=True)
        elif TSlot==ass_tslot and Task=='free' and ass_tslot!=prevTSlot:
            timeSlotStatusWithGap = timeSlotStatusWithGap.append(pd.Series([Employee, ass_tslot, Task, gap, RequiredTask]), ignore_index=True)
            prevTSlot = ass_tslot
#         elif (TSlot==ass_tslot and Task=='free'):
#             timeSlotStatusWithGap = timeSlotStatusWithGap.append(pd.Series([Employee, ass_tslot, Task, gap, RequiredTask]), ignore_index=True)
#             print(Day, ass_tslot, Task, gap, RequiredTask)

timeSlotStatusWithGap = [tuple(x) for x in timeSlotStatusWithGap.to_numpy()]
timeSlotStatusWithGap

[('Employee501595',
  '09:00:00',
  'Downtown Platform',
  -0.8999999999999999,
  'Admin Ordering'),
 ('Employee501595',
  '09:15:00',
  'Downtown Platform',
  -0.20000000000000018,
  'Admin Ordering'),
 ('Employee501595',
  '09:30:00',
  'Downtown Platform',
  0.7000000000000002,
  'Admin Ordering'),
 ('Employee501595',
  '09:45:00',
  'Downtown Platform',
  1.2000000000000002,
  'Admin Ordering'),
 ('Employee501595',
  '10:00:00',
  'Downtown Platform',
  0.6000000000000001,
  'Admin Ordering'),
 ('Employee501595',
  '10:15:00',
  'Downtown Platform',
  -0.6000000000000001,
  'Admin Ordering'),
 ('Employee501595',
  '10:30:00',
  'Downtown Platform',
  -0.8999999999999999,
  'Admin Ordering'),
 ('Employee501595',
  '10:45:00',
  'Downtown Platform',
  -0.8999999999999999,
  'Admin Ordering'),
 ('Employee501595',
  '11:00:00',
  'Downtown Platform',
  -0.2999999999999998,
  'Admin Ordering'),
 ('Employee501595',
  '11:15:00',
  'Downtown Platform',
  -0.20000000000000018,
  'Admin Ord

In [12]:
hourlyGaps = pd.DataFrame()
count =0
totalGap = 0
last3gaps = 0
notFirstRound = False
for Employee, TSlot, Task, Gap, RequiredTask in timeSlotStatusWithGap:
    if count == 0:
        start_slot = TSlot
        firstGap = Gap
        totalGap = Gap
        firstTask = Task
        # print(firstTask)
        if last3gaps != 0 or notFirstRound==True:
            last3gaps = secondGap + thirdGap + fourthGap
            hourlyGaps = hourlyGaps.append(pd.Series([Employee, second_slot, last3gaps+firstGap, [secondTask, thirdTask, fourthTask, firstTask], RequiredTask]), ignore_index=True)
        prevEmployee = Employee
        count +=1
    elif count == 1 and Employee == prevEmployee:
        secondGap = Gap
        second_slot = TSlot
        totalGap = totalGap + Gap
        secondTask = Task
        # print(secondTask)
        if last3gaps != 0 or notFirstRound==True:
            last3gaps = thirdGap + fourthGap + firstGap
            hourlyGaps = hourlyGaps.append(pd.Series([Employee, third_slot, last3gaps+secondGap, [thirdTask, fourthTask, firstTask, secondTask], RequiredTask]), ignore_index=True)
        prevEmployee = Employee
        count +=1
    elif count == 2 and Employee == prevEmployee:
        thirdGap = Gap
        third_slot = TSlot
        totalGap = totalGap + Gap
        thirdTask = Task
        # print(thirdTask)
        prevEmployee = Employee
        if last3gaps != 0 or notFirstRound==True:
            last3gaps = fourthGap + firstGap + secondGap
            hourlyGaps = hourlyGaps.append(pd.Series([Employee, fourth_slot, last3gaps+thirdGap, [fourthTask, firstTask, secondTask, thirdTask], RequiredTask]), ignore_index=True)
        count +=1
    elif count == 3 and Employee ==prevEmployee:
        totalGap = totalGap + Gap
        fourth_slot = TSlot
        fourthGap = Gap
        fourthTask = Task
        # print(fourthTask)
        last3gaps = firstGap + secondGap + thirdGap
        hourlyGaps = hourlyGaps.append(pd.Series([Employee, start_slot, totalGap, [firstTask, secondTask, thirdTask, fourthTask], RequiredTask]), ignore_index=True)
        count = 0
        prevEmployee = Employee
        notFirstRound=True
hourlyGaps = [tuple(x) for x in hourlyGaps.to_numpy()]
hourlyGaps

[('Employee501595',
  '09:00:00',
  0.8000000000000003,
  ['Downtown Platform',
   'Downtown Platform',
   'Downtown Platform',
   'Downtown Platform'],
  'Admin Ordering'),
 ('Employee501595',
  '09:15:00',
  2.3000000000000003,
  ['Downtown Platform',
   'Downtown Platform',
   'Downtown Platform',
   'Downtown Platform'],
  'Admin Ordering'),
 ('Employee501595',
  '09:30:00',
  1.9000000000000004,
  ['Downtown Platform',
   'Downtown Platform',
   'Downtown Platform',
   'Downtown Platform'],
  'Admin Ordering'),
 ('Employee501595',
  '09:45:00',
  0.30000000000000027,
  ['Downtown Platform',
   'Downtown Platform',
   'Downtown Platform',
   'Downtown Platform'],
  'Admin Ordering'),
 ('Employee501595',
  '10:00:00',
  -1.7999999999999998,
  ['Downtown Platform',
   'Downtown Platform',
   'Downtown Platform',
   'Downtown Platform'],
  'Admin Ordering'),
 ('Employee501595',
  '10:15:00',
  -2.6999999999999997,
  ['Downtown Platform',
   'Downtown Platform',
   'Downtown Platform',

In [14]:
employeesWithLowestGapSlots = list()
count = 0
prevEmployee=''
emptyFound=False
for Employee, Start, Gap, Tasks, RequiredTask in hourlyGaps:
    # print(Employee, Start, Tasks, RequiredTask + " coming in")
    if Employee != prevEmployee:
        count = 0
        
    if count == 0:
        employeesWithLowestGapSlots.append((Employee, Start, Gap, Tasks, RequiredTask))
        # print(Employee, Tasks, RequiredTask + " first record of employee")
        count +=1
        prevEmployee = Employee
        if Tasks[0]=='free' and Tasks[1]=='free' and Tasks[2]=='free' and Tasks[3]=='free':
            emptyFound = True
    else:
        # print("Went in as not first record")
        if Tasks[0]=='free' and Tasks[1]=='free' and Tasks[2]=='free' and Tasks[3]=='free':
            emptyFound = True
            print(Employee, Tasks, RequiredTask + " empty found")
            for t in employeesWithLowestGapSlots:
                if t[0] == Employee:
                    employeesWithLowestGapSlots.remove(t)
                    # print(Employee, Tasks, RequiredTask + " not first record of employee")
                    employeesWithLowestGapSlots.append((Employee, Start, Gap, Tasks, RequiredTask))
                    # print(Employee, Start, Gap, Tasks, RequiredTask)
                    prevEmployee = Employee
        if emptyFound == False:
            for t in employeesWithLowestGapSlots:
                if t[0] == Employee and Gap < t[2]:
                    employeesWithLowestGapSlots.remove(t)
                    # print(Employee, Tasks, RequiredTask + " not first record of employee")
                    employeesWithLowestGapSlots.append((Employee, Start, Gap, Tasks, RequiredTask))
                    # print(Employee, Start, Gap, Tasks, RequiredTask)
                    prevEmployee = Employee
                
                    
            
employeesWithLowestGapSlots
        
    

Employee501721 ['free', 'free', 'free', 'free'] Admin StayAtHome empty found


[('Employee501595',
  '14:00:00',
  -3.4999999999999996,
  ['Downtown Platform',
   'Downtown Platform',
   'Downtown Platform',
   'Downtown Platform'],
  'Admin Ordering'),
 ('Employee501721',
  '14:00:00',
  1.7000000000000002,
  ['free', 'free', 'free', 'free'],
  'Admin StayAtHome')]

In [15]:
allocationsToBeRemoved = pd.DataFrame()
allocationsToBeAdded = pd.DataFrame()
columns = ['Employee', 'TSlot', 'Task']
count = 0
for Employee, Time, Gap, Tasks, RequiredTask in employeesWithLowestGapSlots:
    for Task in Tasks:
        if count != 0:
            Time = (datetime.strptime(Time, '%H:%M:%S')+ timedelta(minutes=15)).time().strftime("%H:%M:%S")
        # print(Task)
        allocationsToBeRemoved = allocationsToBeRemoved.append(pd.Series([Employee, Time, Task]), ignore_index=True)
        allocationsToBeAdded = allocationsToBeAdded.append(pd.Series([Employee, Time, RequiredTask]), ignore_index=True)
        count+=1

allocationsToBeRemoved.columns = columns
allocationsToBeAdded.columns = columns
        
        

Downtown Platform
Downtown Platform
Downtown Platform
Downtown Platform
free
free
free
free


In [16]:
allocationsToBeAdded

,Employee,TSlot,Task
0,Employee501595,14:00:00,Admin Ordering
1,Employee501595,14:15:00,Admin Ordering
2,Employee501595,14:30:00,Admin Ordering
3,Employee501595,14:45:00,Admin Ordering
4,Employee501721,14:15:00,Admin StayAtHome
5,Employee501721,14:30:00,Admin StayAtHome
6,Employee501721,14:45:00,Admin StayAtHome
7,Employee501721,15:00:00,Admin StayAtHome


In [17]:
allocationsToBeRemoved

,Employee,TSlot,Task
0,Employee501595,14:00:00,Downtown Platform
1,Employee501595,14:15:00,Downtown Platform
2,Employee501595,14:30:00,Downtown Platform
3,Employee501595,14:45:00,Downtown Platform
4,Employee501721,14:15:00,free
5,Employee501721,14:30:00,free
6,Employee501721,14:45:00,free
7,Employee501721,15:00:00,free
